In [1]:
# Importing required modules
import warnings
import gc
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import torch.optim as optim
import torch.nn as nn
import torch
import cv2
import pandas as pd
import numpy as np
import pathlib
import os
from torchvision.models import resnet50
import wandb
from astropy.io import fits
from imblearn.over_sampling import RandomOverSampler as sampler
# from kaggle_secrets import UserSecretsClient
warnings.filterwarnings('ignore')

/opt/conda/envs/akash/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Seeding to help make results reproduceable
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
def f1_score(preds, targets):
    preds = torch.round(torch.sigmoid(preds))
    tp = torch.sum(targets * preds)
    fp = torch.sum((1 - targets) * preds)
    fn = torch.sum(targets * (1 - preds))
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return f1


In [4]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, X_train, y_train, transform):
        self.root_dir = root_dir
        self.transform = transform
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, index):
        label = self.y_train.iloc[index]
        image_path = self.X_train.iloc[index]['ID']
        hdulist = fits.open(f"lens_finding/SpaceBasedTraining/files/imageEUC_VIS-{'%05d'%image_path}.fits")
        data = hdulist[0].data
        data = (data - np.min(data)) / (np.max(data) - np.min(data))
        data = np.uint8(data * 255)
        data = cv2.cvtColor(data, cv2.COLOR_GRAY2BGR)
        image = self.transform(data)
        self.label = torch.tensor(label)
        return image, self.label

In [5]:
def run():
    torch.cuda.empty_cache()
    seed_everything(SEED)

    #  Loading the train data
    train_df = pd.read_csv(train_csv_path)
    
    # building training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(train_df.drop(columns=['is_lens']), train_df['is_lens'], test_size=0.1, random_state=SEED)
    ros = sampler(random_state=SEED)
    X_train, y_train = ros.fit_resample(X_train, y_train)
    y_train = y_train.astype(float)
    y_valid = y_valid.astype(float)

    print('Data lengths: ', len(X_train), len(X_valid), len(y_train), len(y_valid))
    print("Train distribution: ", y_train.value_counts())

    # Data transforms
    train_transforms = transforms.Compose([transforms.ToPILImage(),
                                            transforms.RandomHorizontalFlip(p=0.5),
                                            transforms.RandomVerticalFlip(p=0.5),
                                            transforms.RandomRotation((-15, 15), interpolation=transforms.InterpolationMode.NEAREST, expand=False, center=None, fill=0),

                                           transforms.ToTensor(),
                                           transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])
    test_transforms = transforms.Compose([transforms.ToPILImage(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])

    # DataLoader
    train_dataset = CustomDataset(root_dir,X_train, y_train,train_transforms)
    valid_dataset = CustomDataset(root_dir,X_valid, y_valid,test_transforms)


    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)


    # Loading the pretrained model here
    if 'efficientnet' in model_name:
        model = EfficientNet.from_pretrained(model_name)
        model._fc = nn.Sequential(
            nn.Linear(mid_features, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )
    else:
        model = resnet50(pretrained = True)
        model._fc = nn.Sequential(
            nn.Linear(mid_features, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )
    # print(model)
    for params in model.parameters():
        params.requires_grad = True

    # Training model
    epoch = 0
    best_s = 0.0
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma_value)
    EPOCHS = num_epoch
    

    while epoch < num_epoch:
        print(f'Epoch: {epoch+1}/{EPOCHS}')

        correct = 0
        total = 0
        losses = []
        val_f1 = 0

        for batch_idx, data in enumerate(tqdm(train_loader, total=len(train_loader))):
            images, targets = data
            images = images.to(device)
            targets = torch.tensor(targets, dtype=torch.float).unsqueeze(1)
            targets = targets.to(device)
            output = model(images)
            output = torch.sigmoid(output)
            loss = criterion(output, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # _, pred = torch.max(output, 1)
            pred = output>= 0.5
            correct += (pred == targets).sum().item()
            total += pred.size(0)
            losses.append(loss.item())
            loss.detach()
            del images, targets, output, loss
            gc.collect()

        train_loss = np.mean(losses)
        train_acc = correct * 100.0 / total
        del losses
        total = 0
        correct = 0
        valid_acc = 0
        losses = []
        f1_scores = []

        with torch.no_grad():
            for batch_idx, data in enumerate(tqdm(valid_loader, total=len(valid_loader))):
                images, targets = data
                images = images.to(device)
                targets = torch.tensor(targets, dtype=torch.float).unsqueeze(1)
                targets = targets.to(device)
                output = model(images)
                output = torch.sigmoid(output)
                loss = criterion(output, targets)
                # _, pred = torch.max(output, 1)
                losses.append(loss.item())
                f1_scores.append(f1_score(output, targets))
                # val_f1 += f1_score(pred, targets)
                pred = output>= 0.5
                correct += (pred == targets).sum().item()
                total += pred.size(0)
        valid_acc = correct/total * 100
        val_loss = np.mean(losses)
        val_f1 = np.mean(np.array([x.cpu().numpy() for x in f1_scores]))
        del losses, f1_scores

        #Saving checkpoint, each time the current valid_acc overshoots the previous best.
        if valid_f1 > best_s:
            checkpoint_name = 'checkpoint_' + str(epoch+1) + '.pth.tar'
            torch.save({'model_state_dict': model.state_dict(),}, checkpoint_name)
            best_s = valid_f1
        print(f'Train Loss: {train_loss}\tTrain Acc: {train_acc}\tLR: {scheduler.get_lr()}\tValid Accuracy: {valid_acc}\tValid F1-score: {val_f1}', end = '\r')
        if(bool_wandb):
            wandb.log({"Train loss": train_loss, "Train Acc": train_acc, "Learning Rate": (scheduler.get_lr()[0]), "Valid Accuracy": valid_acc, "Valid F1-score": val_f1, "Valid loss": val_loss})
        scheduler.step()
        epoch+=1

In [6]:
# This helps make all other paths relative
base_path = pathlib.Path().absolute()
bool_wandb = True

# Model choice
# arg:  efficientnet-b0, efficientnet-b3, resnet50, b5, b6. b7
model_name = 'efficientnet-b7'

# Input of the required hyperparameters
BATCH_SIZE = 128
# 1280, 1536, 2048, 2048, 2304, 2560
mid_features = 2560
learning_rate = 0.002
gamma_value = 0.98

SEED = 42
num_epoch = 1000
num_workers = 2
device = 'cuda:5' if torch.cuda.is_available() else 'cpu'
base_path = 'lens_finding/SpaceBasedTraining'
root_dir = base_path + '/files'
if not os.path.exists(f'{root_dir}'):
    print("Dataset missing.")
train_csv_path = f'{base_path}/classifications.csv'

if bool_wandb:
    wandb.login()
    wandb.init(project="deeplenses", entity="gakash2001")
    wandb.config = {
        'model' : model_name,
        "learning_rate": learning_rate,
        "epochs": num_epoch,
        "batch_size": BATCH_SIZE
    }
run()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gakash2001. Use `wandb login --relogin` to force relogin


Data lengths:  22630 1800 22630 1800
Train distribution:  0.0    11315
1.0    11315
Name: is_lens, dtype: int64
Loaded pretrained weights for efficientnet-b7
Epoch: 1/1000


100%|██████████| 15/15 [00:02<00:00,  6.32it/s]


Epoch: 2/1000.6162866761455428	Train Acc: 73.0755634114008	LR: [0.002]	Valid Accuracy: 63.72222222222222	Valid F1-score: 0.8110722303390503


100%|██████████| 15/15 [00:02<00:00,  6.11it/s]


...

100%|██████████| 15/15 [00:02<00:00,  6.41it/s]


Epoch: 906/1000240762476193703	Train Acc: 94.909412284578	LR: [2.2941853247198206e-11]	Valid Accuracy: 90.33333333333333	Valid F1-score: 0.9331793189048767


100%|██████████| 15/15 [00:02<00:00,  6.43it/s]


Epoch: 907/1000240247512267808	Train Acc: 94.91825011047283	LR: [2.248301618225424e-11]	Valid Accuracy: 90.38888888888889	Valid F1-score: 0.9340344667434692


 15%|█▍        | 26/177 [00:11<01:01,  2.44it/s]

In [ ]:
exit()